# Neural Network

Let's try to "learn" the "exclusive or" function. Unlike "and" or "or", it cannot be learned by a single neuron/perceptron, because it is not linearly seperable.

In [1]:
import numpy as np

For two inputs, xor is true when one of them is true, and false otherwise.

In [2]:
training_inputs = [
    [1, 1],
    [1, 0],
    [0, 1],
    [0, 0]
]

training_labels = [0, 1, 1, 0]

Our main "model". This is the highest-level function here, and will handle:
- Initializing our weights and biases
- Running forward propagation
- Running backward propagation
- Returning the final weights and biases

In [3]:
def model(inputs, labels, learning_rate, iterations):
    parameters = init_params(2, 3, 1)
    return parameters

Initialize our weights and biases. There are a couple interesting things to note here.

First, the input layer doesn't have weights or biases. So if we have a 3-layer network (input, one hidden, and the output), we end up with only two sets of weights/biases.

Second, the number of weights for a layer is the connections between the _previous_ layer and the _current_ one. So if there are 2 inputs and 3 hidden nodes, we have 2 * 3 - or 6 - connections and therefore weights between them.

In [4]:
def init_params(*layer_dimensions):
    num_layers = len(layer_dimensions)
    weights = []
    biases = []
    
    # As discussed above, we end up with `num_layers - 1` sets of weights and biases.
    for l in range(1, num_layers):
        # The number of weights for this layer is the product of the current layer's node count and the previous's.
        w = np.random.randn(layer_dimensions[l], layer_dimensions[l - 1])
        weights.append(w)
        
        # The number of biases for this layer is just the number of nodes.
        b = np.zeros((layer_dimensions[l], 1))
        biases.append(b)
        
    return {
        'weights': weights,
        'biases': biases
    }

We're finally ready to train our model!

In [5]:
parameters = model(training_inputs, training_labels, 0.25, 100)
print(parameters)

{'weights': [array([[ 0.5449696 , -0.83516474],
       [-1.13719156,  0.6335984 ],
       [ 0.48966125,  1.22395043]]), array([[-0.73619146, -0.65498139, -0.992265  ]])], 'biases': [array([[0.],
       [0.],
       [0.]]), array([[0.]])]}
